보스턴 데이터셋의 특징 출력

In [1]:
from sklearn.datasets import fetch_california_housing

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

dataset = fetch_california_housing() # 데이터셋을 불러옴
print(dataset.keys())   # 데이터셋의 키(요소들의 이름)를 출력

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])


# 데이터의 구성요소 확인

In [2]:
import pandas as pd

from sklearn.datasets import fetch_california_housing

dataset = fetch_california_housing()
dataFrame = pd.DataFrame(dataset["data"]) # ❶ 데이터셋의 데이터 불러오기
dataFrame.columns = dataset["feature_names"] # ❷ 특징의 이름 불러오기
dataFrame["target"] = dataset["target"] # ❸ 데이터 프레임에 정답을 추가

print(dataFrame.head()) # ➍ 데이터프레임을 요약해서 출력

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  target  
0    -122.23   4.526  
1    -122.22   3.585  
2    -122.24   3.521  
3    -122.25   3.413  
4    -122.25   3.422  


# 선형회귀를 위한 MLP모델의 설계

In [ ]:
import torch
import torch.nn as nn

from torch.optim.adam import Adam


# ❶ 모델 정의
model = nn.Sequential(
   nn.Linear(13, 100),
   nn.ReLU(),
   nn.Linear(100, 1)
)

X = dataFrame.iloc[:, :13].values # ❷ 정답을 제외한 특징을 X에 입력
Y = dataFrame["target"].values    # 데이터프레임의 target의 값을 추출

batch_size = 100
learning_rate = 0.001

# ❸ 가중치를 수정하기 위한 최적화 정의
optim = Adam(model.parameters(), lr=learning_rate)


# 에포크 반복
for epoch in range(200):

   # 배치 반복
   for i in range(len(X)//batch_size):
       start = i*batch_size      # ➍ 배치 크기에 맞게 인덱스를 지정
       end = start + batch_size


       # 파이토치 실수형 텐서로 변환
       x = torch.FloatTensor(X[start:end])
       y = torch.FloatTensor(Y[start:end])

       optim.zero_grad() # ❺ 가중치의 기울기를 0으로 초기화
       preds = model(x)  # ❻ 모델의 예측값 계산
       loss = nn.MSELoss()(preds, y) # ❼ MSE 손실 계산
       loss.backward()
       optim.step()

   if epoch % 20 == 0:
       print(f"epoch{epoch} loss:{loss.item()}")

epoch0 loss:63.797157287597656
epoch20 loss:45.051475524902344
epoch40 loss:40.557090759277344
epoch60 loss:39.79398727416992
epoch80 loss:39.45023727416992
epoch100 loss:39.383544921875
epoch120 loss:39.04169845581055
epoch140 loss:38.85562515258789
epoch160 loss:38.72293472290039
epoch180 loss:38.49763488769531


# 모델 성능 평가

In [ ]:
prediction = model(torch.FloatTensor(X[0, :13]))
real = Y[0]
print(f"prediction:{prediction.item()} real:{real}")

prediction:25.561281204223633 real:24.0
